In [1]:
# make the screen bigger!
from IPython.display import display, HTML
display(HTML(data=""" <style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 85%; }
    div#maintoolbar-container { width: 99%; } </style> """))

import numpy as np
import os
import pandas as pd
import csv
from datetime import datetime, timedelta
import subprocess

homedir = os.getcwd()


# pandas display options
pd.set_option('display.max_rows', 50)
pd.options.display.float_format = '{:.3f}'.format
pd.set_option('display.precision', 1)    # set number of significant digits to diplay n pandas

In [2]:
colnames_HRU = ['LULC','HRU','GIS','SUB','MGT','MON','AREAkm2','PRECIPmm','SNOFALLmm','SNOMELTmm','IRRmm','PETmm','ETmm','SW_INITmm',
'SW_ENDmm','PERCmm','GW_RCHGmm','DA_RCHGmm','REVAPmm','SA_IRRmm','DA_IRRmm','SA_STmm','DA_STmm','SURQ_GENmm','SURQ_CNTmm','TLOSSmm',
'LATQGENmm','GW_Qmm','WYLDmm','DAILYCN','TMP_AVdgC','TMP_MXdgC','TMP_MNdgC','SOL_TMPdgC','SOLARMJ/m2','SYLDt/ha','USLEt/ha','N_APPkg/ha',
'P_APPkg/ha','NAUTOkg/ha','PAUTOkg/ha','NGRZkg/ha','PGRZkg/ha','NCFRTkg/ha','PCFRTkg/ha','NRAINkg/ha','NFIXkg/ha','F-MNkg/ha','A-MNkg/ha','A-SNkg/ha',
'F-MPkg/ha','AO-LPkg/ha','L-APkg/ha','A-SPkg/ha','DNITkg/ha','NUPkg/ha','PUPkg/ha','ORGNkg/ha','ORGPkg/ha','SEDPkg/ha','NSURQkg/ha','NLATQkg/ha','NO3Lkg/ha',
'NO3GWkg/ha','SOLPkg/ha','P_GWkg/ha','W_STRS','TMP_STRS','N_STRS','P_STRS','BIOMt/ha','LAI','YLDt/ha','BACTPct','BACTLPct','WTABCLIm','WTABSOLm','SNOmm','CMUPkg/ha',
'CMTOTkg/ha','QTILEmm','TNO3kg/ha','LNO3kg/ha','GW_Q_Dmm','LATQCNTmm','TVAPkg/ha']

colnames_RCH = ['whaaaaa', 'RCH',  'GIS',  'MON',  'AREAkm2',  'FLOW_INcms',   'FLOW_OUTcms',  'EVAPcms',  'TLOSScms', 'SED_INtons',   
            'SED_OUTtons',  'SEDCONCmg/L',  'ORGN_INkg',    'ORGN_OUTkg',   'ORGP_INkg',    'ORGP_OUTkg',   'NO3_INkg', 
            'NO3_OUTkg',    'NH4_INkg', 'NH4_OUTkg',    'NO2_INkg', 'NO2_OUTkg',    'MINP_INkg',    'MINP_OUTkg',   'CHLA_INkg',    
            'CHLA_OUTkg',   'CBOD_INkg',    'CBOD_OUTkg',   'DISOX_INkg',   'DISOX_OUTkg',  'SOLPST_INmg',  'SOLPST_OUTmg', 'SORPST_INmg',  
            'SORPST_OUTmg', 'REACTPSTmg',   'VOLPSTmg', 'SETTLPSTmg',   'RESUSP_PSTmg', 'DIFFUSEPSTmg', 'REACBEDPSTmg', 'BURYPSTmg',    
            'BED_PSTmg',    'BACTP_OUTct',  'BACTLP_OUTct', 'CMETAL#1kg',   'CMETAL#2kg',   'CMETAL#3kg',   'TOTNkg',   'TOTPkg',   
            'NO3ConcMg/l',  'WTMPdegc']

colnames_SUB = ['trash', 'SUB',  'GIS',  'MON',  'AREAkm2',  'PRECIPmm', 'SNOMELTmm',    'PETmm',    'ETmm', 'SWmm', 'PERCmm',   'SURQmm',   
            'GW_Qmm',   'WYLDmm',   'SYLDt/ha', 'ORGNkg/ha',    'ORGPkg/ha',    'NSURQkg/ha',   'SOLPkg/ha',    'SEDPkg/ha',    
            'LATQ(mm)', 'LATNO3kg/ha',  'GWNO3kg/ha',   'CHOLAmic/L',   'CBODUmg/L',    'DOXQmg/L', 'TNO3kg/ha',    'QTILEmm',  'TVAPkg/ha']


In [14]:
# Read in HRU file
OUTFILEPATH = r'C:\Users\cshul\Desktop\GitHub_repo_copies\DL_watershed\Q_SWAT_workflow_testing\2_Afono\model\Nuuuli_test1\Scenarios\Default\TxtInOut'

OUTFILE = os.path.join(OUTFILEPATH, "output.hru")

Startdate = "20120101"
OGdate = datetime(year=int(Startdate[0:4]), month=int(Startdate[4:6]), day=int(Startdate[6:8]))



OUTFILE_HRU_yr = os.path.join(OUTFILEPATH, "output.hru")
listo = []

with open(OUTFILE_HRU_yr) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=' ', skipinitialspace=True)
    line_count = 0
    for row in csv_reader:
        listo.append(row)
    da_listo = listo[9:]        # cut off the headder line BS

data = da_listo            # just th

for i in data:
    splitit = i[5].split(".")               # this little blok is because SWAT Fd up the Mon column and stuck it onto the area col, why?
    i[5] = splitit[0]
    i.insert(6, float('0.' + splitit[-1]))  # the bugs and inconsistant formatting in this model are truely mind blowing, the developers of this must be stoned...

df = pd.DataFrame(data, columns=colnames_HRU)
relvnt_df_dumbunits = df[['LULC', 'HRU', 'GIS', 'SUB', 'MGT', 'MON', 'AREAkm2', 'PRECIPmm', 'PETmm', 'ETmm', 'SW_INITmm',
                          'SW_ENDmm', 'PERCmm', 'GW_RCHGmm', 'DA_RCHGmm', 'REVAPmm', 'SA_IRRmm',
                          'DA_IRRmm', 'SA_STmm', 'DA_STmm', 'SURQ_GENmm', 'SURQ_CNTmm', 'TLOSSmm',
                          'LATQGENmm', 'GW_Qmm', 'WYLDmm', 'DAILYCN']]

In [5]:
df.columns


Index(['LULC', 'HRU', 'GIS', 'SUB', 'MGT', 'MON', 'AREAkm2', 'PRECIPmm',
       'SNOFALLmm', 'SNOMELTmm', 'IRRmm', 'PETmm', 'ETmm', 'SW_INITmm',
       'SW_ENDmm', 'PERCmm', 'GW_RCHGmm', 'DA_RCHGmm', 'REVAPmm', 'SA_IRRmm',
       'DA_IRRmm', 'SA_STmm', 'DA_STmm', 'SURQ_GENmm', 'SURQ_CNTmm', 'TLOSSmm',
       'LATQGENmm', 'GW_Qmm', 'WYLDmm', 'DAILYCN', 'TMP_AVdgC', 'TMP_MXdgC',
       'TMP_MNdgC', 'SOL_TMPdgC', 'SOLARMJ/m2', 'SYLDt/ha', 'USLEt/ha',
       'N_APPkg/ha', 'P_APPkg/ha', 'NAUTOkg/ha', 'PAUTOkg/ha', 'NGRZkg/ha',
       'PGRZkg/ha', 'NCFRTkg/ha', 'PCFRTkg/ha', 'NRAINkg/ha', 'NFIXkg/ha',
       'F-MNkg/ha', 'A-MNkg/ha', 'A-SNkg/ha', 'F-MPkg/ha', 'AO-LPkg/ha',
       'L-APkg/ha', 'A-SPkg/ha', 'DNITkg/ha', 'NUPkg/ha', 'PUPkg/ha',
       'ORGNkg/ha', 'ORGPkg/ha', 'SEDPkg/ha', 'NSURQkg/ha', 'NLATQkg/ha',
       'NO3Lkg/ha', 'NO3GWkg/ha', 'SOLPkg/ha', 'P_GWkg/ha', 'W_STRS',
       'TMP_STRS', 'N_STRS', 'P_STRS', 'BIOMt/ha', 'LAI', 'YLDt/ha', 'BACTPct',
       'BACTLPct', 'WTABCLIm',

In [15]:
relvnt_df_dumbunits

,LULC,HRU,GIS,SUB,MGT,MON,AREAkm2,PRECIPmm,PETmm,ETmm,...,DA_IRRmm,SA_STmm,DA_STmm,SURQ_GENmm,SURQ_CNTmm,TLOSSmm,LATQGENmm,GW_Qmm,WYLDmm,DAILYCN
0,FRSE,1,000010001,1,0,1,0.069,288.000,152.500,112.622,...,0.000,1002.245,2001.738,1.668,1.668,0.000,151.462,12.440,166.104,73.206
1,FRSE,2,000010002,1,0,1,0.000,288.000,152.500,113.453,...,0.000,1067.217,2017.744,22.337,22.320,0.000,0.478,131.082,159.171,84.713
2,FRSE,3,000010003,1,0,1,0.172,288.000,152.500,107.054,...,0.000,958.205,2000.068,17.427,17.427,0.000,153.053,0.000,170.502,83.388
3,AGRL,4,000010004,1,0,1,0.002,288.000,152.500,109.605,...,0.000,989.138,2000.333,46.050,46.050,0.000,117.429,0.000,163.586,89.219
4,AGRL,5,000010005,1,0,1,0.002,288.000,152.500,101.386,...,0.000,932.679,2000.000,57.376,57.376,0.000,120.154,0.000,177.529,90.643
5,SWCH,6,000020001,2,0,1,0.000,288.000,152.492,107.310,...,0.000,1051.306,2011.066,5.045,5.045,0.000,68.797,86.484,163.555,77.051
6,FRSE,7,000020002,2,0,1,0.000,288.000,152.492,115.352,...,0.000,1086.373,2022.900,1.992,1.992,0.000,0.819,170.868,180.584,73.626
7,FRSE,8,000020003,2,0,1,0.027,288.000,152.492,112.315,...,0.000,1000.767,2001.440,1.651,1.651,0.000,153.527,9.613,165.234,73.134
8,FRSE,9,000020004,2,0,1,0.099,288.000,152.492,107.051,...,0.000,958.189,2000.068,17.428,17.428,0.000,153.054,0.000,170.504,83.389
9,FRSD,10,000020005,2,0,1,0.002,288.000,152.492,104.974,...,0.000,939.213,2000.003,36.284,36.284,0.000,137.271,0.000,173.556,87.733


In [20]:
import sys
sys.argv

['C:\\Users\\cshul\\Anaconda3\\lib\\site-packages\\ipykernel_launcher.py',
 '-f',
 'C:\\Users\\cshul\\AppData\\Roaming\\jupyter\\runtime\\kernel-62cf965d-94cc-4db2-972f-704f638ad3c6.json']

#  Process output HRU file from Daily run into yearly run

In [ ]:
OUTFILEPATH = r'C:\Users\cshul\Desktop\GitHub_repo_copies\DL_watershed\Q_SWAT_workflow_testing\2_Afono\model\Nuuuli_test1\Scenarios\Default\TxtInOut'
itername    = "iter1"

In [ ]:
""" now open the file.cio file in the folder and change the IPRINT: print code (month, day, year) code to 2 """
os.chdir(OUTFILEPATH)

with open(os.path.join('.', 'file.cio'), 'r') as fin:   # open file 
    data = fin.read().splitlines(True)

Stuff1 = data[0:58]
New_print_time =    '               2    | IPRINT: print code (month, day, year)\n'
Stuff2 = data[59:68]
New_HRU1_vars =     '   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0\n'
stuff3 = data[69]
New_HRU1_print =     '   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0\n'
stuff4 = data[71:]

Stuff1.append(New_print_time)
Stuff1.extend(Stuff2)
Stuff1.append(New_HRU1_vars)
Stuff1.append(stuff3)
Stuff1.append(New_HRU1_print)
Stuff1.extend(stuff4)

with open(os.path.join('.', 'file.cio'), 'w') as fout:  # write new file.cio
    fout.writelines(Stuff1)

In [ ]:
'''run SWAT yearly resolution'''

os.chdir(OUTFILEPATH)
subprocess.call('SWAT.exe', shell=True)

# this modifies the File CIO file to run as Daily again

with open(os.path.join('.', 'file.cio'), 'r') as fin:   # open file 
    data = fin.read().splitlines(True)

    
""" now open the file.cio file in the folder and change the IPRINT: print code back to 1 """
Stuff1 = data[0:58]
New_print_time =    '               1    | IPRINT: print code (month, day, year)\n'
Stuff2 = data[59:68]
New_HRU1_vars =     '   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0\n'
stuff3 = data[69]
New_HRU1_print =     '   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0\n'
stuff4 = data[71:]

Stuff1.append(New_print_time)
Stuff1.extend(Stuff2)
Stuff1.append(New_HRU1_vars)
Stuff1.append(stuff3)
Stuff1.append(New_HRU1_print)
Stuff1.extend(stuff4)

with open(os.path.join('.', 'file.cio'), 'w') as fout:  # write new file.cio
    fout.writelines(Stuff1)

# back to home-dir
os.chdir(homedir)

In [ ]:
OUTFILE_HRU_yr = os.path.join(OUTFILEPATH, "output.hru")
listo = []

with open(OUTFILE_HRU_yr) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=' ', skipinitialspace=True)
    line_count = 0
    for row in csv_reader:
        listo.append(row)
    da_listo = listo[9:]        # cut off the headder line BS

data = da_listo            # just th

for i in data:
    splitit = i[5].split(".")               # this little blok is because SWAT Fd up the Mon column and stuck it onto the area col, why?
    i[5] = splitit[0]
    i.insert(6, float('0.' + splitit[-1]))  # the bugs and inconsistant formatting in this model are truely mind blowing, the developers of this must be stoned...

df = pd.DataFrame(data, columns=colnames_HRU)

Ave_frame = df[df['MON'] == "3"]
Ave_frame = Ave_frame.reset_index(drop=True)    # make index start at 1
 
cols = Ave_frame.columns
Ave_frame[cols] = Ave_frame[cols].apply(pd.to_numeric, errors='coerce', axis=1)  # make the columns numeric not strings 

interestedparamsHRU_mm = ['PRECIPmm','ETmm', 'GW_RCHGmm', 'SURQ_GENmm', 'LATQGENmm', 'GW_Qmm', 'GW_Q_Dmm', 'DA_RCHGmm', 'REVAPmm']
interestedparamsHRU_kgpha = ['NSURQkg/ha','NLATQkg/ha', 'NO3GWkg/ha', 'NO3Lkg/ha', "NCFRTkg/ha", 'N_APPkg/ha', 'F-MNkg/ha', 'DNITkg/ha', 'NUPkg/ha']

area_m2 = Ave_frame['AREAkm2']*1000000
area_ha = Ave_frame['AREAkm2']*100

ColList = []; Val_Lst = []
for i in interestedparamsHRU_mm: 
    p = sum(((Ave_frame[i]*0.001)*area_m2)/365)     # scale by area and convert the mm values to m3/day of water 
    ColList.append(i)  ; Val_Lst.append(p)

for j in interestedparamsHRU_kgpha:
    m = sum(((area_ha*Ave_frame[j])))
    ColList.append(j) ; Val_Lst.append(m)
    
Summary_frameHRU = pd.DataFrame({'a_Variables':ColList,  'b_HRU_value':Val_Lst})
#Summary_frameHRU

Summary_frameHRU.to_csv(os.path.join(OUTFILEPATH, 'Iterations', itername, 'Daily_HRU_summary_frame.csv'))

# Process originally Daily resolution HRU output file (Looong run time)

In [ ]:
OUTFILEPATH = r'C:\Users\cshuler\Desktop\FALU_SWAT_Folder\Nuts_calibration5\GiantPeach_1nut_TESTMOD.Sufi2.SwatCup'
itername = 'MOD_solNO3_5'

OUTFILE = os.path.join(OUTFILEPATH, "output.hru")

Startdate = "20120101"
OGdate = datetime(year=int(Startdate[0:4]), month=int(Startdate[4:6]), day=int(Startdate[6:8]))


# actual column names since they are poorly delimeted in SWAT output file...

listo = []

with open(OUTFILE) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=' ', skipinitialspace=True)
    line_count = 0
    for row in csv_reader:
        listo.append(row)
    data = listo[9:]        # cut off the headder line BS

for i in data:
    splitit = i[5].split(".")               # this little blok is because SWAT Fd up the Mon column and stuck it onto the area col, why?
    i[5] = splitit[0]
    i.insert(6, float('0.' + splitit[-1]))  # the bugs and inconsistant formatting in this model are truely mind blowing, the developers of this must be stoned...    

df = pd.DataFrame(data, columns=colnames_HRU)  # make into dataframe

cols = df.columns
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce', axis=1)  # make the columns numeric not strings |


# just split out the relevant columns to work with 
mm_cols = ['LULC','HRU','GIS','SUB','MON','AREAkm2','PRECIPmm', 'ETmm', 'PERCmm','GW_RCHGmm','DA_RCHGmm','REVAPmm', 'SURQ_GENmm', 'LATQGENmm','GW_Qmm']
kgpha_cols = ['LULC','HRU','GIS','SUB','MON','AREAkm2', 'N_APPkg/ha', 'NCFRTkg/ha','NRAINkg/ha','NFIXkg/ha','F-MNkg/ha','A-MNkg/ha','DNITkg/ha','NUPkg/ha','NSURQkg/ha','NLATQkg/ha','NO3Lkg/ha','LNO3kg/ha', 'NO3GWkg/ha']

mm_frame = df[mm_cols]
kgpha_frame = df[kgpha_cols]


# make values absolute in m3 of water per day or in kgN per day
interested_mm_cols = ['PRECIPmm', 'ETmm', 'PERCmm','GW_RCHGmm','DA_RCHGmm','REVAPmm', 'SURQ_GENmm', 'LATQGENmm','GW_Qmm']
interested_kgpha_cols = ['N_APPkg/ha', 'NCFRTkg/ha','NRAINkg/ha','NFIXkg/ha','F-MNkg/ha','A-MNkg/ha','DNITkg/ha','NUPkg/ha','NSURQkg/ha','NLATQkg/ha','NO3Lkg/ha','LNO3kg/ha', 'NO3GWkg/ha']

mm_frame2 = mm_frame.copy()
for i in interested_mm_cols:
    new_name = i[:-2]+"_m3pd"   #make new column headding 
    mm_frame2[new_name]= (mm_frame[i]*0.001)*(mm_frame['AREAkm2']*1000000)
    del mm_frame2[i]
    
kgpha_frame2 = kgpha_frame.copy()
for i in interested_kgpha_cols:
    new_name = i[:-2]+"_kgpd"   #make new column headding 
    kgpha_frame2[new_name]= (kgpha_frame[i])*(kgpha_frame['AREAkm2']*100)
    del kgpha_frame2[i]


# consolidate the data into yearly averages for each HRU (water data )
Yearly_ave_Nflux_Water = pd.DataFrame(columns = mm_frame2.columns) # bland dataframe to concat to 
HUR_LIST  = mm_frame2['HRU'].unique()   # list of the HRUs
for i in HUR_LIST:
    Isolate_HRU = mm_frame2[mm_frame2['HRU'] == i]
    Average_HRU = Isolate_HRU.groupby('MON').mean()
    Preserve_area = Isolate_HRU['AREAkm2'].mean()
    Consldate_HRU = Average_HRU.groupby('HRU').sum()

    Consldate_HRU['AREAkm2'] = Preserve_area
    
    Yearly_ave_Nflux_Water = pd.concat([Yearly_ave_Nflux_Water, Consldate_HRU])

Yearly_ave_Nflux_Water = Yearly_ave_Nflux_Water.reset_index(drop=False)    # make index start at 1
Yearly_ave_Nflux_Water = Yearly_ave_Nflux_Water.rename(columns={'index': 'HRU_index'})


# consolidate the data into yearly averages for each HRU (Nutrient data )
Yearly_ave_Nflux_Nuts = pd.DataFrame(columns = kgpha_frame2.columns) # bland dataframe to concat to 
HUR_LIST  = kgpha_frame2['HRU'].unique()   # list of the HRUs
for i in HUR_LIST:
    Isolate_HRU = kgpha_frame2[kgpha_frame2['HRU'] == i]
    Average_HRU = Isolate_HRU.groupby('MON').mean()
    Preserve_area = Isolate_HRU['AREAkm2'].mean()
    Consldate_HRU = Average_HRU.groupby('HRU').sum()

    Consldate_HRU['AREAkm2'] = Preserve_area
    
    Yearly_ave_Nflux_Nuts = pd.concat([Yearly_ave_Nflux_Nuts, Consldate_HRU])

Yearly_ave_Nflux_Nuts = Yearly_ave_Nflux_Nuts.reset_index(drop=False)    # make index start at 1
Yearly_ave_Nflux_Nuts = Yearly_ave_Nflux_Nuts.rename(columns={'index': 'HRU_index'})

# put together summary frames now that they are in an average annual format
interestedparamsHRU_mm = ['DA_RCHG_m3pd', 'ET_m3pd', 'PERC_m3pd', 'PRECIP_m3pd', 'REVAP_m3pd', 'GW_Q_m3pd',  'GW_RCHG_m3pd','LATQGEN_m3pd','SURQ_GEN_m3pd']
interestedparamsHRU_kgpha = ['NCFRTkg/_kgpd', 'NFIXkg/_kgpd', 'NLATQkg/_kgpd', 'NO3GWkg/_kgpd', 'NO3Lkg/_kgpd', 'DNITkg/_kgpd',
       'NRAINkg/_kgpd', 'NSURQkg/_kgpd', 'NUPkg/_kgpd', 'N_APPkg/_kgpd']

ColList_Wat = []; Val_Lst_Wat = []
for j in interestedparamsHRU_mm:
    m = sum(((Yearly_ave_Nflux_Water[j]/365)))
    ColList_Wat.append(j) ; Val_Lst_Wat.append(m)

ColList_Nut = []; Val_Lst_Nut = []
for i in interestedparamsHRU_kgpha: 
    p = sum(Yearly_ave_Nflux_Nuts[i])     # scale by area and convert the mm values to m3/day of water 
    ColList_Nut.append(i)  ; Val_Lst_Nut.append(p)

Summary_frame_dailyHRU_water = pd.DataFrame({'a_Variables':ColList_Wat,  'b_HRU_value':Val_Lst_Wat})
Summary_frame_dailyHRU_nuts = pd.DataFrame({'a_Variables':ColList_Nut,  'b_HRU_value':Val_Lst_Nut})

Daily_HRU_summary_frame = Summary_frame_dailyHRU_water.merge(Summary_frame_dailyHRU_nuts, how = 'outer', on='a_Variables')

Daily_HRU_summary_frame.to_csv(os.path.join(OUTFILEPATH, 'Iterations',  itername, 'Daily_HRU_summary_frame.csv'))

# Processing daily resolution .RCH files to get single day measurements 

In [ ]:
OUTFILEPATH =  r'C:\Users\cshuler\Desktop\FALU_SWAT_Folder\Nuts_calibration4\Good_at_cal1_stage\JonJon_1nut_no_flo.Sufi2.SwatCup'


OUTFILE = os.path.join(OUTFILEPATH, "output.rch")


listo = []

with open(OUTFILE) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=' ', skipinitialspace=True)
    line_count = 0
    for row in csv_reader:
        listo.append(row)
    data = listo[9:]        # cut off the headder line BS
    
df = pd.DataFrame(data, columns=colnames_RCH)  # make into dataframe
# clean some trash up
del df['whaaaaa'] 
cols = df.columns
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce', axis=1)  # make the columns numeric not strings 


Startdate = "20120101"
OGdate = datetime(year=int(Startdate[0:4]), month=int(Startdate[4:6]), day=int(Startdate[6:8]))

RCH_pile = {}

for i in range(1,27):
    keynam = "RCH_{}".format(i)
    RCH_pile[keynam] = df[df['RCH'] == i]
    RCH_pile[keynam]  = RCH_pile[keynam] .reset_index(drop=True)    # make index start at 1
    RCH_pile[keynam]  = RCH_pile[keynam] .reset_index(drop=False)    # pull out index value
    RCH_pile[keynam]['date'] = pd.to_datetime(RCH_pile[keynam]['index'], unit='d', origin = OGdate )  # make a date column based on the index as a day index
    
    # use this to pull out specific days of interest from the RCH file

dayofinterest = RCH_pile['RCH_22'][RCH_pile['RCH_22']['date'] == '2014-07-04 00:00:00' ]
dayofinterest
    

# Process NO-obs files. 
### This is getting the 95ppu_g file into an average annual list 

note for SWAT cup the HRU output file col numbers for each variable are: 

NSURQ = col 60
NLATQ = col 61
NO3L = col 62
NO3GW - col 63


In [61]:
OUTFILEPATH_95pp = r'C:\Users\cshuler\Desktop\FALU_SWAT_Folder\Nuts_calibration4\JonJon_1nut_no_flo.Sufi2.SwatCup\SUFI2.OUT'

In [66]:
# this portion opens ut the actual HRU files to calculate the areas and multiply the ppu by Ha
HRU_filepath = OUTFILEPATH_95pp[:-10]
OUTFILE_HRU_95pp = os.path.join(HRU_filepath, "output.hru")
listo = []
with open(OUTFILE_HRU_95pp) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=' ', skipinitialspace=True)
    line_count = 0
    for row in csv_reader:
        listo.append(row)
    data = listo[9:]        # cut off the headder line BS

for i in data:
    splitit = i[5].split(".")               # this little blok is because SWAT Fd up the Mon column and stuck it onto the area col, why?
    i[5] = splitit[0]
    i.insert(6, float('0.' + splitit[-1]))  # the bugs and inconsistant formatting in this model are truely mind blowing, the developers of this must be stoned...

df = pd.DataFrame(data, columns=colnames_HRU) 

Ave_frame = df[df['MON'] == "3"]
Ave_frame = Ave_frame.reset_index(drop=True)    # make index start at 1


# now open the 95ppu_g file for processing
da_95ppu_g_file = os.path.join(OUTFILEPATH_95pp, '95ppu_g_No_Obs.txt')
listo = []
with open(da_95ppu_g_file) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=' ', skipinitialspace=True)
    line_count = 0
    for row in csv_reader:
        listo.append(row)
    data = listo[1:]        # cut off the headder line BS
    
df = pd.DataFrame(data, columns=['Stname', 'L95PPU', 'U95PPU', ''])  # make into dataframe (note for some reason extra space is getting counted as extra column, thus the blank column at the emd)
Ncols = ['L95PPU', 'U95PPU']
df[Ncols] = df[Ncols].apply(pd.to_numeric, errors='coerce', axis=1)

ALL_Vars_95ppu = df.groupby('Stname').mean()
ALL_Vars_95ppu =  ALL_Vars_95ppu.reset_index(drop=False)    # make index start at 1

variable_list = ['NSURQ', 'NLATQ', 'NO3GW', 'NO3L']



All_var_95s = {}
for i in variable_list: 
    i_frame = ALL_Vars_95ppu[ALL_Vars_95ppu['Stname'].str.contains(i)].copy()
   # i_frame['HRU'] = ( i_frame['Stname']).apply(lambda x: int(x.replace('.','_').split('_')[1])) # pulls out the HRU number from the txt
   # i_frame.sort_values('HRU')                   # sort on the HRU number to prepare for area multiplication
    i_frame =  i_frame.reset_index(drop=True)    # apparenetly the multiplication goes by the indes so needs to be corrected  (this might not be needed?)
    i_frame['Area'] = Ave_frame['AREAkm2']*100   # go from KG/ha to KG
    i_frame['L95PPU_kgpd'] = i_frame['Area'] * i_frame['L95PPU']
    i_frame['U95PPU_kgpd'] = i_frame['Area'] * i_frame['U95PPU']
    
    All_var_95s[i] = [i_frame['L95PPU_kgpd'].sum(), i_frame['U95PPU_kgpd'].sum()]

All_var_95s

{'NSURQ': [3308.4344547168366, 4385.5651568444464],
 'NLATQ': [332.35842257405926, 910.87223987687992],
 'NO3GW': [21.17238519053247, 36.312868137705436],
 'NO3L': [3461.8393361006374, 5409.3206174416155]}

In [65]:
All_var_95s

{'NSURQ': [3308.4344547168366, 4385.5651568444464],
 'NLATQ': [332.35842257405926, 910.87223987687992],
 'NO3GW': [21.17238519053247, 36.312868137705436],
 'NO3L': [3461.8393361006374, 5409.3206174416155]}

# (depreciated) YEARLY resolution files 
### to generate yearly files dont forget

- 1: change the run iterations to run the "best" one
- 2: change the file CIO to print at yearly intervals 
- 3: make sure it prints the hru files by changing aly 1s to 0s in the print options
- 4: change the sufi_run bat to print at yearly intervals
-

# (Depreciated) Process yearly resolution output HRU file 

In [64]:
OUTFILEPATH = r'C:\Users\cshuler\Desktop\FALU_SWAT_Folder\Nuts_calibration3\Frankie_1nut_wFlow_SENSitivity.Sufi2.SwatCup'
OUTFILE_HRU_yr = os.path.join(OUTFILEPATH, "output.hru")

# actual column names since they are poorly delimeted in SWAT output file...

listo = []

with open(OUTFILE_HRU_yr) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=' ', skipinitialspace=True)
    line_count = 0
    for row in csv_reader:
        listo.append(row)
    da_listo = listo[9:]        # cut off the headder line BS

data = da_listo            # just th

for i in data:
    splitit = i[5].split(".")               # this little blok is because SWAT Fd up the Mon column and stuck it onto the area col, why?
    i[5] = splitit[0]
    i.insert(6, float('0.' + splitit[-1]))  # the bugs and inconsistant formatting in this model are truely mind blowing, the developers of this must be stoned...

df = pd.DataFrame(data, columns=colnames_HRU)

Ave_frame = df[df['MON'] == "3"]
Ave_frame = Ave_frame.reset_index(drop=True)    # make index start at 1
 
cols = Ave_frame.columns
Ave_frame[cols] = Ave_frame[cols].apply(pd.to_numeric, errors='coerce', axis=1)  # make the columns numeric not strings 

interestedparamsHRU_mm = ['PRECIPmm','ETmm', 'GW_RCHGmm', 'SURQ_GENmm', 'LATQGENmm', 'GW_Qmm', 'GW_Q_Dmm', 'DA_RCHGmm', 'REVAPmm']
interestedparamsHRU_kgpha = ['NSURQkg/ha','NLATQkg/ha', 'NO3GWkg/ha', 'NO3Lkg/ha', "NCFRTkg/ha", 'N_APPkg/ha', 'F-MNkg/ha', 'DNITkg/ha', 'NUPkg/ha']

area_m2 = Ave_frame['AREAkm2']*1000000
area_ha = Ave_frame['AREAkm2']*100

ColList = []; Val_Lst = []
for i in interestedparamsHRU_mm: 
    p = sum(((Ave_frame[i]*0.001)*area_m2)/365)     # scale by area and convert the mm values to m3/day of water 
    ColList.append(i)  ; Val_Lst.append(p)

for j in interestedparamsHRU_kgpha:
    m = sum(((area_ha*Ave_frame[j])))
    ColList.append(j) ; Val_Lst.append(m)
    
Summary_frameHRU = pd.DataFrame({'a_Variables':ColList,  'b_HRU_value':Val_Lst})
Summary_frameHRU

,a_Variables,b_HRU_value
0,PRECIPmm,20218.131
1,ETmm,7626.481
2,GW_RCHGmm,2414.722
3,SURQ_GENmm,4355.163
4,LATQGENmm,5630.409
5,GW_Qmm,1774.760
6,GW_Q_Dmm,630.377
7,DA_RCHGmm,639.731
8,REVAPmm,356.757
9,NSURQkg/ha,559.778


# Process YEARLY resolution output SUB file 

In [ ]:
OUTFILE = os.path.join(OUTFILEPATH, "output.sub")




listo = []
with open(OUTFILE) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=' ', skipinitialspace=True)
    line_count = 0
    for row in csv_reader:
        listo.append(row)
    data = listo[9:]        # cut off the headder line BS

for i in data:
    splitit = i[3].split(".")               # this little blok is because SWAT Fd up the Mon column and stuck it onto the area col, why?
    i[3] = splitit[0]
    i.insert(4, float('0.' + splitit[-1]))  # the bugs and inconsistant formatting in this model are truely mind blowing, the developers of this must be stoned...
    
df = pd.DataFrame(data, columns=colnames_SUB)
Ave_frame = df[df['MON'] == "3"]
Ave_frame = Ave_frame.reset_index(drop=True)    # make index start at 1
cols = Ave_frame.columns
Ave_frame[cols] = Ave_frame[cols].apply(pd.to_numeric, errors='coerce', axis=1)  # make the columns numeric not strings 

Ave_frame

interestedparamsHRU_mm = ['PRECIPmm','ETmm', 'PERCmm', 'SURQmm', 'LATQ(mm)', 'GW_Qmm'  ]
interestedparamsHRU_kgpha = ['NSURQkg/ha', 'LATNO3kg/ha',  'GWNO3kg/ha' ]

area_m2 = Ave_frame['AREAkm2']*1000000
area_ha = Ave_frame['AREAkm2']*100

ColList = []; Val_Lst = []
for i in interestedparamsHRU_mm: 
    p = sum(((Ave_frame[i]*0.001)*area_m2)/365)     # scale by area and convert the mm values to m3/day of water 
    ColList.append(i)  ; Val_Lst.append(p)

for j in interestedparamsHRU_kgpha:
    m = sum(((area_ha*Ave_frame[j])))
    ColList.append(j) ; Val_Lst.append(m)
    

Summary_frameSUB = pd.DataFrame({'a_Variables':ColList,  'b_HRU_value':Val_Lst})



In [ ]:
# merge and chec cohesivness of output SUB and Output HRU based on yearly files

yearly_HRU_SUB_summary_frame= Summary_frameSUB.merge(Summary_frameHRU, how = 'outer', on='a_Variables')
#Output_stats.to_csv(os.path.join(OUTFILEPATH, 'Output_stats.csv'))